# Trainieren eines neuronalen Netzwerks zur Klassifikation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

## Iris Datensatz

Der Iris Datensatz (siehe https://en.wikipedia.org/wiki/Iris_flower_data_set) enthält je 50 Beispiele für jede der drei Iris Blumen Arten (Iris setosa, Iris virginica and Iris versicolor). Gemessen wurden je die Breite und die Länge des Kelch- und Kronenblattes (Sepal and Petal). Auf Grundlage dieser Messung soll die Art der Blume bestimmt werden.

In [ ]:
from sklearn.datasets import load_iris
iris_dataset = load_iris()
iris_dataset

In [ ]:
iris_features = iris_dataset['data']
iris_targets = iris_dataset['target']
iris_target_names = iris_dataset['target_names']
iris_target_names

In [ ]:
iris_targets

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(iris_features[iris_targets==2, 2], iris_features[iris_targets==2, 3], "g^", label=iris_target_names[2])
plt.plot(iris_features[iris_targets==1, 2], iris_features[iris_targets==1, 3], "bs", label=iris_target_names[1])
plt.plot(iris_features[iris_targets==0, 2], iris_features[iris_targets==0, 3], "yo", label=iris_target_names[0])

plt.xlabel("Petal length", fontsize=14)
plt.ylabel("Petal width", fontsize=14)
plt.legend(loc="center left", fontsize=14)
plt.axis([0, 7, 0, 3.5])

plt.show()

## Modell-Definition

In [ ]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense 

In [ ]:
model = Sequential([
    Dense(4, activation='relu', dtype=tf.float64),
    Dense(3, activation='softmax', dtype=tf.float64) 
])

In [ ]:
model.compile(
    optimizer='sgd', 
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    iris_features, 
    iris_targets, 
    epochs=20, 
    batch_size=2,
    validation_split=0.2
)

In [ ]:
hist_df = history.history #pd.DataFrame(history.history)
plt.figure(figsize=(14, 4))
plt.plot(hist_df['val_accuracy'], label='val_accuracy')
plt.plot(hist_df['accuracy'], label='accuracy')
plt.legend()
plt.show()

In [ ]:
model.summary()

In [ ]:
idx = 20
iris_features[idx:idx+1]

In [ ]:
# mit der predict Methode lässt sich das Modell verwenden um Vorhersagen zu machen
idx = 20
model.predict(iris_features[idx:idx+1])

In [ ]:
# np.argmax lässt sich so die wahrscheinlichste Klasse auswählen:
np.argmax(model.predict(iris_features[idx:idx+1]))

In [ ]:
idx = 46
print('Predicted label:', iris_target_names[np.argmax(model.predict(iris_features[idx:idx+1]))])
print('True label:', iris_target_names[iris_targets[idx]])

## Hier versuchen wir das gleiche Problem mit Entscheidungsbäumen anzugehen:

### Voting Classifiers

Der Voting Classifier kombiniert die Vorhersagen dreier Klassifizierer: Decision Tree, Random Forest und eine SVM

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [ ]:
log_clf = DecisionTreeClassifier(random_state=42)
rnd_clf = RandomForestClassifier(random_state=42)
svm_clf = SVC(random_state=42)

voting_clf = VotingClassifier(estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
                             voting='hard')

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris_features, iris_targets,test_size=0.2, random_state=42)
voting_clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

# Übung 1:
* Betrachte den vorliegenden Datensatz
* Um was für Daten handelt es sich?
* Trainiere ein Modell auf dem Datensatz und Werte die Accuracy aus.

In [ ]:
from tensorflow.keras.datasets import cifar10
(train_data, train_target), (test_data,test_target) = cifar10.load_data()